In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
con = sqlite3.connect("./database.sqlite")

In [3]:
def read_table(table_name, con):
    return pd.read_sql_query(f"SELECT * FROM {table_name}", con)

In [4]:
def remove_unused_columns_matches(matches):
    # print("Initial shape of matches dataframe: ", matches.shape)
    count_nulls = matches.isnull().sum()
    columns_to_remove = count_nulls[count_nulls > 0]
    matches.drop(columns=columns_to_remove.index, inplace=True)
    matches.drop(columns=["id", "match_api_id"], inplace=True)

    # Check the shape is the correct one
    assert matches.shape == (25979, 9)
    # print("Final shape of matches dataframe: ", matches.shape)


In [5]:
def convert_date_format(matches):
    matches["date"] = matches["date"].astype("datetime64[ns]")
    
    assert matches.dtypes["date"] == np.dtype("<M8[ns]")

In [6]:
def add_country_to_matches(matches, countries):
    # Transform column 'country_id' to 'country' using the name of the countries
    countries_dict = dict([(id, name) for id, name in zip(countries["id"], countries["name"])])
    matches["country"] = matches["country_id"].map(countries_dict)
    matches.drop(columns=["country_id"], inplace=True)

    # Check the final dataframe has a column 'country' and does not have a column 'country_id'
    assert matches.shape == (25979, 9)
    assert "country" in matches.columns
    assert "country_id" not in matches.columns

In [7]:
def add_league_column_to_matches(matches, leagues):
    leagues_dict = dict([id, name] for id, name in zip(leagues["id"], leagues["name"]))
    matches["league"] = matches["league_id"].map(leagues_dict)
    matches.drop(columns=["league_id"], inplace=True)

    # Check the final dataframe has a column 'league' and does not have a column 'league_id'
    assert matches.shape == (25979, 9)
    assert "league" in matches.columns
    assert "league_id" not in matches.columns

In [8]:
def add_team_names_to_matches(matches, teams):
    teams_dict = dict([id, name] for id, name in zip(teams["team_api_id"], teams["team_long_name"]))
    matches["home_team"] = matches["home_team_api_id"].map(teams_dict)
    matches["away_team"] = matches["away_team_api_id"].map(teams_dict)
    matches.drop(columns=["home_team_api_id", "away_team_api_id"], inplace=True)
    
    # Check the final dataframe has two new columns 'home_team' and 'away_team'
    assert matches.shape == (25979, 9)
    assert {"home_team", "away_team"}.issubset(matches.columns)
    assert not {"home_team_api_id", "away_team_api_id"}.issubset(matches.columns)

Work with matches

In [9]:
def data_wrangling_matches(matches, countries, leagues, teams):
    remove_unused_columns_matches(matches)
    convert_date_format(matches)
    add_country_to_matches(matches, countries)
    add_league_column_to_matches(matches, leagues)
    add_team_names_to_matches(matches, teams)
    

In [10]:
countries = read_table(table_name="Country", con=con)
leagues = read_table(table_name="League", con=con)
matches = read_table(table_name="Match", con=con)
teams = read_table(table_name="Team", con=con)

In [11]:
matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_wrangling_matches(matches, countries, leagues, teams)
matches

,season,stage,date,home_team_goal,away_team_goal,country,league,home_team,away_team
0,2008/2009,1,2008-08-17,1,1,Belgium,Belgium Jupiler League,KRC Genk,Beerschot AC
1,2008/2009,1,2008-08-16,0,0,Belgium,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren
2,2008/2009,1,2008-08-16,0,3,Belgium,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht
3,2008/2009,1,2008-08-17,5,0,Belgium,Belgium Jupiler League,KAA Gent,RAEC Mons
4,2008/2009,1,2008-08-16,1,3,Belgium,Belgium Jupiler League,FCV Dender EH,Standard de Liège
...,...,...,...,...,...,...,...,...,...
25974,2015/2016,9,2015-09-22,1,0,Switzerland,Switzerland Super League,FC St. Gallen,FC Thun
25975,2015/2016,9,2015-09-23,1,2,Switzerland,Switzerland Super League,FC Vaduz,FC Luzern
25976,2015/2016,9,2015-09-23,2,0,Switzerland,Switzerland Super League,Grasshopper Club Zürich,FC Sion
25977,2015/2016,9,2015-09-22,0,0,Switzerland,Switzerland Super League,Lugano,FC Zürich


In [13]:
matches.isna().sum()

season            0
stage             0
date              0
home_team_goal    0
away_team_goal    0
country           0
league            0
home_team         0
away_team         0
dtype: int64

In [14]:
matches.dtypes

season                    object
stage                      int64
date              datetime64[ns]
home_team_goal             int64
away_team_goal             int64
country                   object
league                    object
home_team                 object
away_team                 object
dtype: object

In [18]:
matches.to_csv("./matches.csv", index=False)